In [ ]:
# Mocking API Responses & S3 for Tests
# -----------------------------------
import json
import boto3
import pandas as pd
from io import StringIO
from moto import mock_s3
from unittest.mock import patch
import requests

# 1️⃣ Mock API call
@patch("requests.get")
def fetch_employee_data(mock_get):
    # Simulate API JSON response
    mock_get.return_value.json.return_value = {
        "employees": [
            {"id": 1, "name": "Aisha", "role": "Analyst"},
            {"id": 2, "name": "Ravi", "role": "Manager"}
        ]
    }
    response = requests.get("https://fakeapi.company.com/employees")
    return response.json()

print("🌐 Fetching mock API data...")
api_data = fetch_employee_data()
print(json.dumps(api_data, indent=2))

# 2️⃣ Convert JSON → DataFrame
df = pd.DataFrame(api_data["employees"])
print("\n📊 Converted DataFrame:")
print(df)

# 3️⃣ Mock S3 upload
@mock_s3
def upload_to_s3(df):
    s3 = boto3.client("s3", region_name="us-east-1")
    bucket_name = "mock-bucket"
    s3.create_bucket(Bucket=bucket_name)
    
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    
    s3.put_object(Bucket=bucket_name, Key="employees.csv", Body=csv_buffer.getvalue())
    print(f"✅ Uploaded employees.csv to {bucket_name} (mocked S3).")

upload_to_s3(df)

# 4️⃣ Validate Upload
@mock_s3
def validate_s3():
    s3 = boto3.client("s3", region_name="us-east-1")
    bucket_name = "mock-bucket"
    s3.create_bucket(Bucket=bucket_name)
    objects = s3.list_buckets()
    print(f"🪣 Mock S3 Buckets: {[b['Name'] for b in objects['Buckets']]}")

validate_s3()

"""
This script shows:
- How to mock an API (simulate JSON response)
- How to mock S3 uploads with moto
- Safe and fast testing of ETL-like flow
"""
